## Import necessary packages

In [1]:
import pandas as pd

In [2]:
import numpy as np

## Import the datasets

In [3]:
products = pd.read_csv("./products.csv")
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [4]:
order_prior=pd.read_csv("./order_products__prior.csv")
order_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [5]:
dept=pd.read_csv("./departments.csv")
dept

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


## Merge the Datasets

In [6]:
merge=pd.merge(products, order_prior, on="product_id")
merge.head()

,product_id,product_name,aisle_id,department_id,order_id,add_to_cart_order,reordered
0,1,Chocolate Sandwich Cookies,61,19,1107,7,0
1,1,Chocolate Sandwich Cookies,61,19,5319,3,1
2,1,Chocolate Sandwich Cookies,61,19,7540,4,1
3,1,Chocolate Sandwich Cookies,61,19,9228,2,0
4,1,Chocolate Sandwich Cookies,61,19,9273,30,0


In [7]:
merge=merge.groupby("department_id").size().reset_index(name="dep_count")
merge

,department_id,dep_count
0,1,2236432
1,2,36291
2,3,1176787
3,4,9479291
4,5,153696
5,6,269253
6,7,2690129
7,8,97724
8,9,866627
9,10,34573


In [8]:
merge=merge.merge(dept[["department_id","department"]], on="department_id")
merge

,department_id,dep_count,department
0,1,2236432,frozen
1,2,36291,other
2,3,1176787,bakery
3,4,9479291,produce
4,5,153696,alcohol
5,6,269253,international
6,7,2690129,beverages
7,8,97724,pets
8,9,866627,dry goods pasta
9,10,34573,bulk


In [9]:
bins = [0, merge["dep_count"].quantile(0.25), merge["dep_count"].quantile(0.75), max(merge["dep_count"])]
names = ["low","medium","high"]

merge['Department_consumption'] = pd.cut(merge['dep_count'], bins, labels=names)
merge




,department_id,dep_count,department,Department_consumption
0,1,2236432,frozen,high
1,2,36291,other,low
2,3,1176787,bakery,medium
3,4,9479291,produce,high
4,5,153696,alcohol,low
5,6,269253,international,low
6,7,2690129,beverages,high
7,8,97724,pets,low
8,9,866627,dry goods pasta,medium
9,10,34573,bulk,low


In [17]:
merge=merge[merge["Department_consumption"]=="medium"]

In [10]:
merge["Department_consumption"]

0       high
1        low
2     medium
3       high
4        low
5        low
6       high
7        low
8     medium
9        low
10    medium
11    medium
12    medium
13    medium
14    medium
15      high
16    medium
17    medium
18      high
19    medium
20       low
Name: Department_consumption, dtype: category
Categories (3, object): [low < medium < high]

In [11]:
prod_subset= pd.merge(products,merge, on="department_id")
prod_subset

,product_id,product_name,aisle_id,department_id,dep_count,department,Department_consumption
0,1,Chocolate Sandwich Cookies,61,19,2887550,snacks,high
1,16,Mint Chocolate Flavored Syrup,103,19,2887550,snacks,high
2,25,Salted Caramel Lean Protein & Fiber Bar,3,19,2887550,snacks,high
3,32,Nacho Cheese White Bean Chips,107,19,2887550,snacks,high
4,41,Organic Sourdough Einkorn Crackers Rosemary,78,19,2887550,snacks,high
...,...,...,...,...,...,...,...
49683,43769,Organic Pearled Barley,68,10,34573,bulk,low
49684,45682,Organic Turkish Apricots,68,10,34573,bulk,low
49685,46889,Organic Brown Basmati Rice,68,10,34573,bulk,low
49686,47489,Organic Brown Jasmine Rice,68,10,34573,bulk,low


In [19]:
order_prior=pd.read_csv("./order_products__prior.csv")
order_prior

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [12]:
order_sub=pd.merge(order_prior, prod_subset[["product_name","product_id"]], on="product_id")
order_sub

,order_id,product_id,add_to_cart_order,reordered,product_name
0,2,33120,1,1,Organic Egg Whites
1,26,33120,5,0,Organic Egg Whites
2,120,33120,13,0,Organic Egg Whites
3,327,33120,5,1,Organic Egg Whites
4,390,33120,28,1,Organic Egg Whites
...,...,...,...,...,...
32434484,3265099,43492,3,0,Gourmet Burger Seasoning
32434485,3361945,43492,19,0,Gourmet Burger Seasoning
32434486,3267201,33097,2,0,Piquillo & Jalapeno Bruschetta
32434487,3393151,38977,32,0,Original Jerky


In [13]:
g = order_sub.groupby('order_id')

a=np.arange(g.ngroups)
np.random.shuffle(a)

temp=order_sub[g.ngroup().isin(a[:250000])]# change 2 to what you need :-) 

In [14]:
(temp.groupby('order_id').agg('count')).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
product_id,250000.0,10.077944,7.511117,1.0,5.0,8.0,14.0,116.0
add_to_cart_order,250000.0,10.077944,7.511117,1.0,5.0,8.0,14.0,116.0
reordered,250000.0,10.077944,7.511117,1.0,5.0,8.0,14.0,116.0
product_name,250000.0,10.077944,7.511117,1.0,5.0,8.0,14.0,116.0


In [15]:
temp

,order_id,product_id,add_to_cart_order,reordered,product_name
2,120,33120,13,0,Organic Egg Whites
15,1097,33120,14,1,Organic Egg Whites
17,1300,33120,1,1,Organic Egg Whites
34,4254,33120,10,1,Organic Egg Whites
42,6173,33120,1,1,Organic Egg Whites
...,...,...,...,...,...
32434385,3087873,30654,23,0,Translucent Bronzing Tint
32434415,3215939,36059,19,0,Acai Organic Freeze-Dried Acai Powder
32434422,2872631,22709,5,0,Gluten Free Rosemary Truffle Kale Chips
32434429,2898029,23269,4,0,Sheer Volume Shampoo & Conditioner


In [16]:
temp=temp.sort_values('order_id')

In [17]:

del temp["reordered"]

In [18]:
temp

,order_id,product_id,add_to_cart_order,product_name
3028033,12,38050,10,All Natural Boneless Skinless Chicken Breasts
3027323,12,26910,8,100% Juice No Sugar Added Apple
3027296,12,3164,7,100% Juice No Added Sugar Orange Tangerine
3022819,12,34335,6,Ruby Red Grapefruit Juice
3018147,12,37215,5,100% Cranberry Juice
...,...,...,...,...
13946269,3421071,32293,4,Organic Bunny Fruit Tropical Treat Fruit Snacks
30853926,3421071,26919,1,"Fabric Softener Sheets, Blue Eucalyptus & Lave..."
12222438,3421071,35921,3,Organic Large Grade A Brown Eggs
25101304,3421071,34270,5,"Cold-pressed, Deliciously Hydrating Watermelon..."


In [51]:
temp.to_csv("./order_medium.csv", index=False)

In [27]:
order_medium.dropna()

,order_id,product_id,add_to_cart_order,reordered,product_name
0,2,9327,3,0,Garlic Powder
1,63,9327,3,0,Garlic Powder
6,5810,9327,3,0,Garlic Powder
10,7456,9327,10,0,Garlic Powder
12,8353,9327,8,0,Garlic Powder
...,...,...,...,...,...
9066378,3116306,1066,5,0,Chicken with Noodles Low Sodium Ready To Serve...
9066381,3188686,34211,3,0,Greek Pitted Kalamata Olives
9066383,3228642,6786,6,0,Anjou Pear Hand Soap
9066384,3235061,7726,8,0,Blue 1000 Mg Vitamin C Triple Berry Fizzy Drin...


In [17]:
merge.to_csv("./dep_consumption.csv", index=False)